## Cleaning Recipe Dataset


In [2]:
import pandas as pd
df = pd.read_csv(
    r"C:\Users\User\Desktop\ELO2-Smart-Pantry-Manager\the_app\data\Recipe_Dataset.csv"
)

In [3]:
import numpy as np

In [4]:
df.head()

,Unnamed: 0,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients
0,0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher..."
1,1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"['2 large egg whites', '1 pound new potatoes (..."
2,2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"['1 cup evaporated milk', '1 cup whole milk', ..."
3,3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"['1 (¾- to 1-pound) round Italian loaf, cut in..."
4,4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"['1 teaspoon dark brown sugar', '1 teaspoon ho..."


In [15]:
df['ingredient_count'] = df['Ingredients'].apply(len)

In [16]:
df['ingredient_count']

0        828
1        251
2        289
3        558
4        218
        ... 
13496    413
13497    405
13498    271
13499    209
13500    762
Name: ingredient_count, Length: 13501, dtype: int64

In [17]:
df["instruction_steps"] = df["Instructions"].apply(
    lambda x: len([s for s in str(x).split(".") if s.strip()])
)


In [19]:
def extract_keywords(title):
    """Extract main keywords from recipe title"""
    # Common words to exclude
    exclude = {"with", "and", "the", "for", "from", "or"}
    words = str(title).lower().split()
    keywords = [w for w in words if w not in exclude and len(w) > 3]
    return keywords[:3]  # Return top 3 keywords


df["keywords"] = df["Title"].apply(extract_keywords)


In [23]:
def is_vegetarian(ingredients_list):
    """Check if recipe is vegetarian based on ingredients"""
    # Non-vegetarian keywords
    non_veg_keywords = [
        "chicken",
        "beef",
        "pork",
        "lamb",
        "turkey",
        "duck",
        "veal",
        "fish",
        "salmon",
        "tuna",
        "shrimp",
        "prawn",
        "lobster",
        "crab",
        "anchovy",
        "anchovies",
        "bacon",
        "ham",
        "sausage",
        "chorizo",
        "meat",
        "steak",
        "ribs",
        "wings",
        "drumstick",
        "thigh",
        "cod",
        "halibut",
        "snapper",
        "sardine",
        "clam",
        "oyster",
        "mussels",
        "scallop",
        "octopus",
        "squid",
        "gelatin",
    ]

    # Convert all ingredients to lowercase string
    ingredients_text = " ".join([str(ing).lower() for ing in ingredients_list])
    for keyword in non_veg_keywords:
        if keyword in ingredients_text:
            return False

    return True


In [29]:
df["vegetarian"] = df["Ingredients"].apply(is_vegetarian)

print(f"Sample metadata:")
print(df[["Title", "ingredient_count", "instruction_steps", "vegetarian"]].head(5))
print(
    f"\nVegetarian recipes: {df['vegetarian'].sum()} out of {len(df)} ({df['vegetarian'].sum()/len(df)*100:.1f}%)"
)


Sample metadata:
                                               Title  ingredient_count  \
0  Miso-Butter Roast Chicken With Acorn Squash Pa...               828   
1                    Crispy Salt and Pepper Potatoes               251   
2                        Thanksgiving Mac and Cheese               289   
3                 Italian Sausage and Bread Stuffing               558   
4                                       Newton's Law               218   

   instruction_steps  vegetarian  
0                 37        True  
1                  6        True  
2                 14        True  
3                 18        True  
4                  5        True  

Vegetarian recipes: 13501 out of 13501 (100.0%)


In [35]:
print("\n" + "=" * 50)
print("Step 12: Final summary...")
print(f"\nFinal shape: {df.shape}")
print(f"\nColumn names: {df.columns.tolist()}")
print(f"\nData types:\n{df.dtypes}")
print(f"\nMissing values:\n{df.isnull().sum()}")
print(f"\nSample statistics:")
print(df[["ingredient_count", "instruction_steps"]].describe())

# Step 13: Save cleaned data
print("\n" + "=" * 50)
print("Step 13: Saving cleaned data...")
df.to_csv("cleaned_recipes.csv", index=False)
print("Data saved to 'cleaned_recipes.csv'")



Step 12: Final summary...

Final shape: (13306, 10)

Column names: ['Unnamed: 0', 'Title', 'Ingredients', 'Instructions', 'Image_Name', 'Cleaned_Ingredients', 'ingredient_count', 'instruction_steps', 'keywords', 'vegetarian']

Data types:
Unnamed: 0              int64
Title                  object
Ingredients            object
Instructions           object
Image_Name             object
Cleaned_Ingredients    object
ingredient_count        int64
instruction_steps       int64
keywords               object
vegetarian               bool
dtype: object

Missing values:
Unnamed: 0             0
Title                  1
Ingredients            0
Instructions           4
Image_Name             0
Cleaned_Ingredients    0
ingredient_count       0
instruction_steps      0
keywords               0
vegetarian             0
dtype: int64

Sample statistics:
       ingredient_count  instruction_steps
count      13306.000000       13306.000000
mean         419.513077          15.150909
std          201.

In [ ]:
print("\n" + "=" * 50)
print("Step 9: Checking for duplicates...")
duplicates = df.duplicated(subset=["Title"], keep="first")
print(f"Duplicate recipes found: {duplicates.sum()}")

if duplicates.sum() > 0:
    print("Removing duplicates...")
    df = df.drop_duplicates(subset=["Title"], keep="first")
    print(f"Shape after removing duplicates: {df.shape}")

# Step 10: Reset index
print("\n" + "=" * 50)
print("Step 10: Resetting index...")
df = df.reset_index(drop=True)



Step 9: Checking for duplicates...
Duplicate recipes found: 0

Step 10: Resetting index...


In [5]:
df.columns

Index(['Unnamed: 0', 'Title', 'Ingredients', 'Instructions', 'Image_Name',
       'Cleaned_Ingredients'],
      dtype='object')

In [6]:
df.shape

(13501, 6)

In [9]:
pip install openpyxl


   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   ---------------------------------------- 2/2 [openpyxl]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
df2 = pd.read_excel(r'C:\Users\User\Desktop\ELO2-Smart-Pantry-Manager\the_app\data\pantry_data.xlsx')

In [11]:
df2.head()

,Product,Expiry Date,Category,Days Left
0,Milk,2025-10-25,Dairy,NaN
1,Bread,2025-10-21,Bakery,NaN
2,Apples,2025-10-22,Fruits,NaN
3,Rice,2026-03-10,Grains,NaN
4,Chicken,2025-10-23,Meat,NaN


In [13]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Product      12 non-null     object        
 1   Expiry Date  12 non-null     datetime64[ns]
 2   Category     12 non-null     object        
 3   Days Left    2 non-null      float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 516.0+ bytes


In [14]:
df2['Product'].value_counts()

Product
Milk            1
Bread           1
Apples          1
Rice            1
Chicken         1
Yogurt          1
Eggs            1
Pasta           1
Tomato Sauce    1
Cheese          1
Mango juice     1
Brown bread     1
Name: count, dtype: int64